In [175]:
import pandas as pd
df = pd.read_csv('./data/users.csv')
df.head()

,user_id,name,city,street
0,102,Cyprian Oleksiewicz,Konin,pl. Zdrojowa 876
1,103,Kacper Wasiuk,Konin,aleja Boczna 12/47
2,104,Maurycy Dziób,Radom,al. Powstańców Wielkopolskich 605
3,105,Natasza Michalkiewicz,Gdynia,ulica Bankowa 22/96
4,106,Oskar Rohde,Szczecin,aleja Jana Sobieskiego 62/99


In [176]:
df = df.drop(labels=['name'], axis=1)
df.head()

,user_id,city,street
0,102,Konin,pl. Zdrojowa 876
1,103,Konin,aleja Boczna 12/47
2,104,Radom,al. Powstańców Wielkopolskich 605
3,105,Gdynia,ulica Bankowa 22/96
4,106,Szczecin,aleja Jana Sobieskiego 62/99


In [177]:
def preprocess_street(street: str):
    split = street.split()
    split[0] = split[0][:2] + '.'
    split.pop()
    return ' '.join(split)

df['street'] = df['street'].apply(preprocess_street)
df.head()
df.to_csv('users_preprocessed.csv', index=False)

In [178]:
df = pd.read_csv('./data/sessions.csv')
df.head()

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-01-07T08:05:23,102.0,1277.0,VIEW_PRODUCT,0,NaN
1,100001,2021-01-07T08:09:16,102.0,1276.0,VIEW_PRODUCT,0,NaN
2,100001,2021-01-07T08:13:32,102.0,1276.0,BUY_PRODUCT,0,20001.0
3,100002,2021-01-02T02:02:07,102.0,1049.0,VIEW_PRODUCT,0,NaN
4,100002,2021-01-02T02:04:41,102.0,1050.0,VIEW_PRODUCT,0,NaN


In [179]:
df = df.drop(labels=['timestamp', 'offered_discount', 'session_id'], axis=1)
df.head()

,user_id,product_id,event_type,purchase_id
0,102.0,1277.0,VIEW_PRODUCT,NaN
1,102.0,1276.0,VIEW_PRODUCT,NaN
2,102.0,1276.0,BUY_PRODUCT,20001.0
3,102.0,1049.0,VIEW_PRODUCT,NaN
4,102.0,1050.0,VIEW_PRODUCT,NaN


In [180]:
option = ['BUY_PRODUCT']
df = df[df['event_type'].isin(option)]
df = df.drop(labels=['event_type'], axis=1)
df.head()

,user_id,product_id,purchase_id
2,102.0,1276.0,20001.0
6,102.0,1048.0,20002.0
10,102.0,1067.0,20003.0
16,102.0,1316.0,20004.0
19,102.0,1033.0,20005.0


In [181]:
df = df.dropna(subset=['user_id', 'purchase_id'])
df.to_csv('sessions_preprocessed.csv', index=False)

In [182]:
df = pd.read_csv('./data/deliveries.csv')
df.head()


,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-01-07T08:13:32,2021-01-10T17:01:58.118577,360.0
1,20002,2021-01-02T02:07:38,2021-01-04T06:44:02.125108,516.0
2,20003,2021-01-03T17:26:51,2021-01-07T19:23:07.707973,360.0
3,20004,2021-01-05T17:16:55,2021-01-07T15:21:38.391150,516.0
4,20005,2021-01-05T14:40:06,2021-01-08T13:15:22.984316,NaN


In [183]:
def process_company(company):
    return 'C' + str(int(company))

df = df.dropna()
df['delivery_company'] = df['delivery_company'].apply(process_company)
df.head()

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-01-07T08:13:32,2021-01-10T17:01:58.118577,C360
1,20002,2021-01-02T02:07:38,2021-01-04T06:44:02.125108,C516
2,20003,2021-01-03T17:26:51,2021-01-07T19:23:07.707973,C360
3,20004,2021-01-05T17:16:55,2021-01-07T15:21:38.391150,C516
5,20006,2021-01-07T14:35:53,2021-01-10T08:09:09.974023,C620


In [184]:
def cut_milliseconds(date):
    return date[0:-7]

df['delivery_timestamp'] = df['delivery_timestamp'].apply(cut_milliseconds)

df.head()

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-01-07T08:13:32,2021-01-10T17:01:58,C360
1,20002,2021-01-02T02:07:38,2021-01-04T06:44:02,C516
2,20003,2021-01-03T17:26:51,2021-01-07T19:23:07,C360
3,20004,2021-01-05T17:16:55,2021-01-07T15:21:38,C516
5,20006,2021-01-07T14:35:53,2021-01-10T08:09:09,C620


In [185]:
from datetime import datetime
pattern = '%Y-%m-%dT%H:%M:%S'

def get_month(row, column):
    date: datetime = datetime.strptime(row[column], pattern)
    return date.month


def get_day(row, column):
    date: datetime = datetime.strptime(row[column], pattern)
    return date.day


def get_hour(row, column):
    date: datetime = datetime.strptime(row[column], pattern)
    return date.hour

def get_weekday(row, column):
    date: datetime = datetime.strptime(row[column], pattern)
    return date.strftime("%A")

df['purchase_month'] = df.apply(lambda row: get_month(row, 'purchase_timestamp'), axis=1)
df['purchase_day'] = df.apply(lambda row: get_day(row, 'purchase_timestamp'), axis=1)
df['purchase_hour'] = df.apply(lambda row: get_hour(row, 'purchase_timestamp'), axis=1)
df['purchase_weekday'] = df.apply(lambda row: get_weekday(row, 'purchase_timestamp'), axis=1)

df['delivery_month'] = df.apply(lambda row: get_month(row, 'delivery_timestamp'), axis=1)
df['delivery_day'] = df.apply(lambda row: get_day(row, 'delivery_timestamp'), axis=1)
df['delivery_hour'] = df.apply(lambda row: get_hour(row, 'delivery_timestamp'), axis=1)
df['delivery_weekday'] = df.apply(lambda row: get_weekday(row, 'delivery_timestamp'), axis=1)

df.head()

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,purchase_month,purchase_day,purchase_hour,purchase_weekday,delivery_month,delivery_day,delivery_hour,delivery_weekday
0,20001,2021-01-07T08:13:32,2021-01-10T17:01:58,C360,1,7,8,Thursday,1,10,17,Sunday
1,20002,2021-01-02T02:07:38,2021-01-04T06:44:02,C516,1,2,2,Saturday,1,4,6,Monday
2,20003,2021-01-03T17:26:51,2021-01-07T19:23:07,C360,1,3,17,Sunday,1,7,19,Thursday
3,20004,2021-01-05T17:16:55,2021-01-07T15:21:38,C516,1,5,17,Tuesday,1,7,15,Thursday
5,20006,2021-01-07T14:35:53,2021-01-10T08:09:09,C620,1,7,14,Thursday,1,10,8,Sunday


In [186]:
def extract_delivery_time_in_hours(row):
    purchase: datetime = datetime.strptime(row['purchase_timestamp'], pattern)
    delivery: datetime = datetime.strptime(row['delivery_timestamp'], pattern)
    delivery_times = delivery - purchase
    return int(delivery_times.total_seconds()/3600)

df['shipping_time_in_hours'] = df.apply(lambda row: extract_delivery_time_in_hours(row), axis=1)
positive = df['shipping_time_in_hours'] > 0
df = df[positive]
df.head()

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company,purchase_month,purchase_day,purchase_hour,purchase_weekday,delivery_month,delivery_day,delivery_hour,delivery_weekday,shipping_time_in_hours
0,20001,2021-01-07T08:13:32,2021-01-10T17:01:58,C360,1,7,8,Thursday,1,10,17,Sunday,80
1,20002,2021-01-02T02:07:38,2021-01-04T06:44:02,C516,1,2,2,Saturday,1,4,6,Monday,52
2,20003,2021-01-03T17:26:51,2021-01-07T19:23:07,C360,1,3,17,Sunday,1,7,19,Thursday,97
3,20004,2021-01-05T17:16:55,2021-01-07T15:21:38,C516,1,5,17,Tuesday,1,7,15,Thursday,46
5,20006,2021-01-07T14:35:53,2021-01-10T08:09:09,C620,1,7,14,Thursday,1,10,8,Sunday,65


In [187]:
df = df.drop(labels=['delivery_timestamp', 'purchase_timestamp'], axis=1)
df.to_csv('deliveries_preprocessed.csv', index=False)
df.head()


,purchase_id,delivery_company,purchase_month,purchase_day,purchase_hour,purchase_weekday,delivery_month,delivery_day,delivery_hour,delivery_weekday,shipping_time_in_hours
0,20001,C360,1,7,8,Thursday,1,10,17,Sunday,80
1,20002,C516,1,2,2,Saturday,1,4,6,Monday,52
2,20003,C360,1,3,17,Sunday,1,7,19,Thursday,97
3,20004,C516,1,5,17,Tuesday,1,7,15,Thursday,46
5,20006,C620,1,7,14,Thursday,1,10,8,Sunday,65
